In [5]:
import os
import copy
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import graphviz
import pydot

model_classify = Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')])
model_classify.compile(optimizer=Adam(learning_rate=0.005), 
    loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_classify.fit()
#plot_model(model_classify, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
window = 200
slide = 0.1
train_test = 0.8

def make_windows_full_dataset(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq']
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        sample = frame.loc[(i*jump):(i*jump)+(window-1), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        means = [np.mean(sample[col]) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        sample2.append(means)
        if frame['Still'][(i*jump)+199] == 1:
            yval = 0
        if frame['Walk'][(i*jump)+199] == 1:
            yval = 1
        if frame['Run'][(i*jump)+199] == 1:
            yval = 2
        if frame['Cycle'][(i*jump)+199] == 1:
            yval = 3
        
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test

def reshaper(frame):
    data_2d = frame.reshape(-1, frame.shape[-1])
    scaler = StandardScaler()
    whitened_data_2d = scaler.fit_transform(data_2d)
    whitened_frame = whitened_data_2d.reshape(frame.shape)
    return whitened_frame


In [9]:
frame = pd.read_csv('Complete_dataset_PCA_Fourier.csv')
train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_full_dataset(window, slide, frame, train_test)
tf_sparse_train = np.array(sparse_train)
tf_labels_train = np.array(labels_train)
tf_sparse_test = np.array(sparse_test)
tf_labels_test = np.array(labels_test)

In [49]:
print(tf_sparse_train.shape)
whitened_tf_sparse_train = reshaper(tf_sparse_train[1:])

model_classify = Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(4, activation='softmax')])
model_classify.compile(optimizer=Adam(learning_rate=0.03), 
    loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_classify.fit(whitened_tf_sparse_train, tf_labels_train[1:], epochs=8)
whitened_tf_sparse_test = reshaper(tf_sparse_test[1:])
model_classify.evaluate(whitened_tf_sparse_test, tf_labels_test[1:])

(5792, 5, 6)
Epoch 1/8
181/181 [==============================] - 1s 2ms/step - loss: 0.2914 - accuracy: 0.8948
Epoch 2/8
181/181 [==============================] - 0s 2ms/step - loss: 0.2297 - accuracy: 0.9064
Epoch 3/8
181/181 [==============================] - 0s 2ms/step - loss: 0.1947 - accuracy: 0.9194
Epoch 4/8
181/181 [==============================] - 0s 2ms/step - loss: 0.2874 - accuracy: 0.9057
Epoch 5/8
181/181 [==============================] - 0s 2ms/step - loss: 0.2573 - accuracy: 0.8992
Epoch 6/8
181/181 [==============================] - 0s 2ms/step - loss: 0.2294 - accuracy: 0.9026
Epoch 7/8
181/181 [==============================] - 0s 2ms/step - loss: 0.2517 - accuracy: 0.9024
Epoch 8/8
45/45 [==============================] - 0s 745us/step - loss: 2.9981 - accuracy: 0.8885


[2.99806547164917, 0.8884993195533752]

In [52]:
def grid_search_hyperparam():
    epochs = [i+1 for i in range(10)]
    learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
    dropouts = [0.1, 0.2, 0.5]
    histories = {}
    for epoch in epochs:
        for lr in learning_rates:
            for drop in dropouts:
                model_classify = Sequential([
                    Flatten(input_shape=(5, 6)),
                    Dense(64, activation='relu'),
                    Dropout(drop),
                    Dense(128, activation='relu'),
                    Dropout(drop),
                    Dense(64, activation='relu'),
                    Dropout(drop),
                    Dense(4, activation='softmax')
                ])
                model_classify.compile(optimizer=Adam(learning_rate=lr), 
                               loss='sparse_categorical_crossentropy', 
                               metrics=['accuracy'])
                history = model_classify.fit(whitened_tf_sparse_train, tf_labels_train[1:], epochs=epoch)
                whitened_tf_sparse_test = reshaper(tf_sparse_test[1:])
                performance = model_classify.evaluate(whitened_tf_sparse_test, tf_labels_test[1:])

                histories[f'Training Accuracy {epoch}, {lr}, {drop}'] = {
                    'Epochs': epoch,
                    'LR' : lr,
                    'Dropout' : drop,
                    'Training Accuracy': history.history['accuracy'],
                    'Test Accuracy': performance[1]
                }
    return histories

results = grid_search_hyperparam()
epochs = [i+1 for i in range(10)]
learning_rates = [0.1, 0.05, 0.03, 0.02, 0.01, 0.005, 0.002, 0.001]
dropouts = [0.1, 0.2, 0.5]
dropout = [[],[],[]]
learning_params = [[],[],[],[],[],[],[],[]]
epoch = [[],[],[],[],[],[],[],[],[],[]]
for key, value in results.items():
    epoch_key = value['Epochs']
    epoch[epochs.index(epoch_key)].append(value['Test Accuracy'])
    drop_key = value['Dropout']
    dropout[dropouts.index(drop_key)].append(value['Test Accuracy'])
    lr_key = value['LR']
    learning_params[learning_rates.index(lr_key)].append(value['Test Accuracy'])




45/45 [==============================] - 0s 733us/step - loss: 0.4901 - accuracy: 0.8597
Epoch 1/2
181/181 [==============================] - 1s 2ms/step - loss: 0.9982 - accuracy: 0.7403
Epoch 2/2
45/45 [==============================] - 0s 748us/step - loss: 1.4082 - accuracy: 0.3766
Epoch 1/2
181/181 [==============================] - 1s 2ms/step - loss: 1.1115 - accuracy: 0.7821
Epoch 2/2
45/45 [==============================] - 0s 753us/step - loss: 0.3759 - accuracy: 0.9516
Epoch 1/2
181/181 [==============================] - 1s 2ms/step - loss: 1.4747 - accuracy: 0.6071
Epoch 2/2
45/45 [==============================] - 0s 747us/step - loss: 1.7371 - accuracy: 0.3766
Epoch 1/2
181/181 [==============================] - 1s 2ms/step - loss: 0.3764 - accuracy: 0.8714
Epoch 2/2
45/45 [==============================] - 0s 679us/step - loss: 0.2617 - accuracy: 0.9446
Epoch 1/2
181/181 [==============================] - 1s 2ms/step - loss: 0.4779 - accuracy: 0.8508
Epoch 2/2
45/45 [===

In [53]:
for k in range(3):
    print('Dropout '+str(dropouts[k])+' Max: '+str(max(dropout[k])))
for i in range(10):
    print('Epochs '+str(i+1)+' Max: '+str(max(epoch[i])))
for j in range(8):
    print('LR '+str(learning_rates[j])+' Max: '+str(max(learning_params[j])))

Dropout 0.1 Max: 0.9852734804153442
Dropout 0.2 Max: 0.9754558205604553
Dropout 0.5 Max: 0.9460027813911438
Epochs 1 Max: 0.9852734804153442
Epochs 2 Max: 0.9775596261024475
Epochs 3 Max: 0.9410939812660217
Epochs 4 Max: 0.9719495177268982
Epochs 5 Max: 0.9775596261024475
Epochs 6 Max: 0.969845712184906
Epochs 7 Max: 0.9593268036842346
Epochs 8 Max: 0.9368863701820374
Epochs 9 Max: 0.9754558205604553
Epochs 10 Max: 0.9551191926002502
LR 0.1 Max: 0.9852734804153442
LR 0.05 Max: 0.9775596261024475
LR 0.03 Max: 0.9754558205604553
LR 0.02 Max: 0.9775596261024475
LR 0.01 Max: 0.9410939812660217
LR 0.005 Max: 0.9719495177268982
LR 0.002 Max: 0.9446002840995789
LR 0.001 Max: 0.9389901757240295
